In [87]:
import tensorflow as tf
import numpy as np
import scipy.io.wavfile
import os
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [88]:
# data preprocessing
seq_length = 800
merge_factor = 4#temporal resolution = merge_factor/16000 #discrimination_window = merge_factor*seq_length/16000
n_train = 180; n_test = 180;
num_channels = 2
input_size = num_channels*merge_factor #n_channels*merge_factor
num_classes = 5 #n_directions
offset = 64
train_x = []; train_t = []
test_x = []; test_t = []
for i in range(num_classes):
    angle = 0+(45*i)
    temp_data = scipy.io.wavfile.read('speech2/'+'d'+str(angle)+'.wav')[1][:]
    for j in range(n_train+n_test):
        if j%2 != 1:
            train_x.append(temp_data[offset+(j*seq_length*merge_factor):offset+((j+1)*seq_length*merge_factor)])
            train_t.append(i)
    #for j in range(n_train,n_train+n_test):
        else:
            test_x.append(temp_data[offset+(j*seq_length*merge_factor):offset+((j+1)*seq_length*merge_factor)])
            test_t.append(i)

train_x = np.array(train_x); train_t = np.array(train_t)
train_x = train_x.reshape(n_train*num_classes,seq_length,input_size)
test_x = np.array(test_x); test_t = np.array(test_t)
test_x = test_x.reshape(n_test*num_classes,seq_length,input_size)

In [89]:
# hyperparameters
x = tf.placeholder(tf.float32, [None,seq_length, input_size])
labels = tf.placeholder(tf.int32, [None])
i_size = 40
h_size = 40
g_size = 40
learning_rate = tf.placeholder(tf.float32,[])

In [90]:
# parameters
# r_i = np.sqrt(6.0/(input_size + context_size))
# r_c = np.sqrt(6.0/(context_size + context_size))
# r_y = np.sqrt(6.0/(context_size + num_classes))
sdev = 0.1   
params = {
    'Wxi': tf.Variable(tf.random_normal([input_size, i_size],stddev=sdev)),
    'Wih': tf.Variable(tf.random_normal([i_size,h_size],stddev=sdev)),
    'Whg': tf.Variable(tf.random_normal([h_size, g_size],stddev = sdev)),
    'Wgg': tf.Variable(tf.random_normal([g_size, g_size],stddev = sdev)), #initial_value=tf.cast(np.eye(g_size),dtype=tf.float32)
    'Wgy': tf.Variable(tf.random_normal([g_size,num_classes],stddev = sdev)),
    'bi': tf.Variable(tf.zeros([i_size])),
    'bh': tf.Variable(tf.zeros([h_size])),
    'bg': tf.Variable(tf.zeros([g_size])),
    'by': tf.Variable(tf.zeros([num_classes]))
}

In [91]:
# architecture
def RNN(x,params):
    batch_size = tf.shape(x)[0]
    g = tf.zeros([batch_size,g_size])
    for t in range(seq_length):
        i = tf.tanh(tf.add(tf.matmul(x[:,t,:],params['Wxi']),params['bi']))
        h = tf.tanh(tf.add(tf.matmul(i,params['Wih']),params['bh']))
        g = tf.tanh(tf.add(tf.add(tf.matmul(g,params['Wgg']),tf.matmul(h,params['Whg'])),params['bg']))
    y = tf.add(tf.matmul(g,params['Wgy']),params['by'])
    return y

In [92]:
# training strategy
y = RNN(x,params)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(y,labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cost)
capped_grads_and_vars = [(tf.clip_by_value(gv[0],-1.,1.), gv[1]) for gv in grads_and_vars]
capped_optimizer = optimizer.apply_gradients(capped_grads_and_vars)
saver = tf.train.Saver()

In [93]:
sess.run(tf.initialize_all_variables())

In [101]:
n_iter = 0
while True:
    _,cost_ = sess.run([capped_optimizer,cost],feed_dict={x:train_x,labels:train_t,learning_rate:0.01})
    print('iter'+str(n_iter)+' cost:'+str(cost_))
    if n_iter % 20 == 0:
        y_train = sess.run(y,feed_dict={x:train_x})
        print 'train pf = ' + str((sum(np.argmax(y_train,axis=1) == train_t) + 0.0)/(n_train*num_classes))
        y_test = sess.run(y,feed_dict={x:test_x})
        print 'test pf = ' + str((sum(np.argmax(y_test,axis=1) == test_t) + 0.0)/(n_test*num_classes))+'\n'
#         mypath = 'vars/iter'+str(n_iter)
#         if not os.path.isdir(mypath):
#            os.makedirs(mypath)
#         myfile = mypath+'/'+str(n_iter)
#         saver.save(sess,myfile,write_meta_graph=False)
    n_iter += 1

iter0 cost:0.885075
train pf = 0.693333333333
test pf = 0.228888888889

iter1 cost:0.857339
iter2 cost:0.838114
iter3 cost:0.834426
iter4 cost:0.828988
iter5 cost:0.821341
iter6 cost:0.816901
iter7 cost:0.816618
iter8 cost:0.81227
iter9 cost:0.809235
iter10 cost:0.806966
iter11 cost:0.804642
iter12 cost:0.800788
iter13 cost:0.797357
iter14 cost:0.796251
iter15 cost:0.795851
iter16 cost:0.793312
iter17 cost:0.79248
iter18 cost:0.795659
iter19 cost:0.791253
iter20 cost:0.789797
train pf = 0.731111111111
test pf = 0.233333333333

iter21 cost:0.790303
iter22 cost:0.788218
iter23 cost:0.784712
iter24 cost:0.787817
iter25 cost:0.787184
iter26 cost:0.787239
iter27 cost:0.784208
iter28 cost:0.779648
iter29 cost:0.777028
iter30 cost:0.777102
iter31 cost:0.788015
iter32 cost:0.777032
iter33 cost:0.776093
iter34 cost:0.770755
iter35 cost:0.76636
iter36 cost:0.766998
iter37 cost:0.766042
iter38 cost:0.762144
iter39 cost:0.761424
iter40 cost:0.761771
train pf = 0.751111111111
test pf = 0.2322222222

KeyboardInterrupt: 

In [86]:
sess.close()

In [80]:
# new test data
seq_length = 800
merge_factor = 4#temporal resolution = merge_factor/16000 #discrimination_window = merge_factor*seq_length/16000
n_test = 360;
num_channels = 2
input_size = num_channels*merge_factor #n_channels*merge_factor
num_classes = 5 #n_directions
offset = 64
test_x = []; test_t = []
for i in range(num_classes):
    angle = 0+(45*i)
    temp_data = scipy.io.wavfile.read('speech2/'+'d'+str(angle)+'.wav')[1]
    #temp_data = scipy.io.wavfile.read('test.wav')[1]
    for j in range(n_test):
            test_x.append(temp_data[offset+(j*seq_length*merge_factor):offset+((j+1)*seq_length*merge_factor)])
            test_t.append(i)

test_x = np.array(test_x); test_t = np.array(test_t)
test_x = test_x.reshape(n_test*num_classes,seq_length,input_size)
y_test = sess.run(y,feed_dict={x:test_x})
a = np.argmax(y_test,axis=1)
np.set_printoptions(threshold=np.inf)
print a
print 'test pf = ' + str((sum(a == test_t) + 0.0)/(n_test*num_classes))+'\n'

[2 1 1 0 0 0 2 2 4 0 4 0 1 1 1 1 1 1 0 4 2 2 2 0 2 2 1 1 1 1 0 0 1 1 1 0 1
 1 0 0 4 1 2 1 1 0 2 1 2 1 1 1 0 1 1 4 2 1 1 1 0 0 1 0 0 4 0 2 0 1 1 2 1 1
 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 2 2 4 1 1 1 1 1 1 0 0
 1 0 1 2 1 0 1 0 4 1 0 1 1 1 0 0 0 1 1 2 1 4 0 1 1 1 1 2 0 1 1 4 0 0 1 0 4
 2 2 1 0 1 0 2 0 1 4 0 1 0 1 0 1 1 2 0 1 0 0 1 1 4 2 0 0 0 0 1 0 2 4 1 1 1
 1 1 1 0 0 1 1 2 0 1 1 1 0 1 1 1 1 0 1 2 0 0 2 1 1 0 2 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 0 0 4 1 4 0 0 1 1 1 1 1 0 1 2 1 1 0 0 1 4 2 1 1 1 2 1 1 1 1 0 1 1
 0 0 0 4 0 0 1 1 1 0 1 0 0 2 1 0 1 0 1 1 0 1 1 1 1 4 4 0 0 0 2 1 0 0 2 1 2
 1 0 1 0 1 1 1 1 2 0 1 1 1 1 1 2 4 2 0 1 1 1 1 2 0 0 4 2 0 0 0 0 2 4 1 0 1
 4 1 0 0 2 2 0 1 1 0 1 0 1 0 2 0 0 0 0 0 0 0 1 0 1 1 1 4 1 0 4 1 1 1 0 0 0
 1 1 0 1 2 2 1 1 1 1 1 1 0 1 2 1 0 0 1 1 1 0 1 2 0 4 1 1 1 1 1 1 1 0 0 0 1
 1 0 1 4 4 0 2 1 1 2 0 1 0 0 0 2 2 0 0 0 0 2 1 1 4 0 1 1 1 0 1 1 1 0 4 1 1
 0 0 0 1 1 1 1 0 1 1 0 2 0 0 1 1 2 0 0 1 0 1 0 1 1 1 1 1 1 0 1 0 2 1 1 1 0
 4 1 1 1 1 1 1 1 1 0 4 2 

In [107]:
var = params['Wxi'].eval()

[[ 0.48005694  0.15055427  0.88199216  1.16462886  0.54344243 -1.10566723
  -0.08513057 -1.08320999  0.34082413  0.7670604  -0.13694742 -0.15869716
   0.28625828 -0.21132289 -0.03741002  1.24234259  1.05640435  0.13938001
   0.25449252 -1.03470349 -0.09650949 -0.98656207 -0.20474197  0.26544335
  -0.49741796 -0.69012272  0.49664998  1.30719697  0.0131028   0.32043955
   0.73096704 -0.48393908 -1.33944154 -0.01340605  0.34093472 -0.94560033
   0.27994648 -0.07995617 -0.34937885  0.10677422]
 [ 0.64371884  0.18987203  0.70021105  0.94224602  0.22143304 -0.92571706
  -0.36325708  0.27240863 -0.83976859  0.71396554  0.27501214  0.85625666
  -0.11009715  0.45200041  0.78372234  1.23681641 -0.31305283 -0.05921368
   0.59527242 -0.49125737 -0.15702532 -0.11698069 -1.07355261  0.07230431
  -0.41204128  0.0147009   0.70562679 -0.28617612 -0.30787349  0.05203527
   0.55861336  0.6588034  -0.16618143 -0.90671206 -0.39303878 -0.96107173
  -0.51395875  0.81070322  0.29636702  0.00724911]
 [ 0.70554

In [109]:
import matplotlib.pyplot as plt
plt.imshow(var,cmap='Greys')
plt.colorbar()
plt.show()